Title: Enviroment

Author: Chad Wood

Date: 06 June 2022

Modified By: Chad Wood

Description: This is documentation for the class Enviroment.

## Class Enviroment

This class was designed to provide an agent with an enviroment to train on. It also provides a method to return the action space of a given node. I actually have two implimentations of class Enviroment, due to the potential for using PyTorch Geometries in a future update. That implimentation is made available for view in this notebook's bottom cell.

In [2]:
from polypocket import StreetMap
from polypocket import generate

### Generating a polygon for testing class

In [6]:
center = (-121.885254, 37.335796)
dist = 350 # Meters

# Creates polygon with given dimensions
poly = StreetMap(center, dist)

n_houses = 3 # Houses
n_gasstations = 2 # Gas stations

# Instructs class to generate random coordinate points within polygon
objects = generate(streetmap=poly, houses=n_houses, gasstations=n_gasstations).get('osmids')

### Class Source Code

In [477]:
import numpy as np
import geopandas as gpd


class Enviroment():
    
    def __init__(self, G, objects):
        '''
        This class was designed to provide an agent with an enviroment to train on. It also 
        provides a method to return the action space of a given node.
        
        params: 
                G:                  MultiDiGraph object generated via polypocket.StreetMap.
                objects:            Dict object containing indexs of OSMIDs that represent a house or gasstation
                
        attributes: 
                indexer:            Returns dict where keys=osmids and values=labels (label encoding)
                houses:             Returns Int64Index of s hape (n, ); [house_osmid]
                gasstations:        Returns Int64Index of shape (n, ); [gasstation_osmid]
                norm_houses:        Returns array of shape (n, ); [house_label]
                norm_gasstations:   Returns array of shape (n, ); [gasstation_label]
                nodes:              Returns array of shape (n, 2); [node-osmid, dict(attributes)]
                edges:              Returns GeoDataFrame of edges multiindexed by osmid, includes attributes length and geometry
                edges_arr:          Returns array of shape (n, 3); [edgeStart_osmid, edgeEnd_osmid, length]
                norm_nodes:         Returns array of shape (n, ); [node_label]
                norm_edges:         Returns array of shape (n, 3); [edgeStart_label, edgeEnd_label, length]
                flagged_edges:      Returns array of shape (n, 5); [(edges_arr+), bool_is_house*1, bool_is_gasstation*1]
                norm_flagged_edges: Returns array of shape (n, 5); [(norm_edges+), bool_is_house*1, bool_is_gasstation*1]
        
        methods: 
                get_actions(self, node_id): Returns array of shape (n, 2); [neighbor_node_id, length]
        '''
        # Maps each node to an index based on order of appearance
        def indexer(nodes):
            nodes_index = list(range(len(nodes)))
            return dict(zip(nodes, nodes_index))
        

        # Filters edges for only necessary attributes, returns GeoDataFrame
        def filter_edges(edges):
            attribute_dict = dict()
            for node, connection, data_dict in edges:
                edge = (node, connection)
                attribute_dict.update({edge:data_dict})  
                
            edges = gpd.GeoDataFrame(attribute_dict).T
            edges['length'] = edges['length'].astype(float)
            
            return edges[['length', 'geometry']]
        

        # Converts edges to array
        def edges_to_array(self):
            edges_arr = np.array([i for i in self.edges.index])
            attributes = self.edges['length'].to_numpy().reshape(-1,1)
            return np.append(edges_arr, attributes, axis=1)

        
        # Normalizes node IDs
        def normalize_nodes(self):
            return np.arange(len(self.nodes)).reshape(-1,1)
        
        
        # Normalized edge IDs and attributes
        def normalize_edges(self):
            df = self.edges.rename(index=self.indexer)
            df = df.drop(columns=['geometry']) 
            return np.append(np.array([*df.index]), df.values, axis=1)
        
        
        def edge_flags(self):
            # Mask of edges that travel to a target or gasstation
            edges_to_targets     = np.isin(self.norm_edges[:,1], self.norm_houses).reshape(-1,1)*1
            edges_to_gasstations = np.isin(self.norm_edges[:,1], self.norm_gasstations).reshape(-1,1)*1
            
            # Appends for use as flags to better generalize the data
            edge_flags = np.append(edges_to_targets, edges_to_gasstations, axis=1)

            return edge_flags
        
        
        # Creates dict maping a standard index to each node
        self.indexer = indexer(G.nodes)
        
        # Enviroment objects
        self.houses      = objects.get('houses')
        self.gasstations = objects.get('gasstations')
        self.norm_houses      = np.array(list(map(self.indexer.get, self.houses)))
        self.norm_gasstations = np.array(list(map(self.indexer.get, self.gasstations)))
        
        # Nodes and edges
        self.nodes = np.array(G.nodes(data=True))
        self.edges = filter_edges(G.edges(data=True))
        self.edges_arr = edges_to_array(self)
        
        # Normalized node and edge IDs
        self.norm_nodes = normalize_nodes(self)
        self.norm_edges = normalize_edges(self)
        
        # Flags what edges travel to a target or gasstation using a mask column
        self.flagged_edges      = np.append(self.edges_arr, edge_flags(self), axis=1)
        self.norm_flagged_edges = np.append(self.norm_edges, edge_flags(self), axis=1)
    
    
    # Returns available edges from a node
    def get_actions(self, node_id):   
        # Checks normalized edges first
        if node_id in self.norm_edges[:,0]:
            return self.norm_edges[self.norm_edges[:,0] == node_id][:,1:]
        # Checks default edges
        elif node_id in self.edges.index:
            return self.edges.loc[node_id]
        
        else:
            raise KeyError(f'node_id "{node_id}" was not found.')

### Example Usage
The code below demonstrates now simple it now is to collect an enviroment using a graph and objects. The array returned from norm_flagged_edges is of shape(n, 5) where:

    column1 = edgeStart,
    column2 = edgeEnd,
    column3 = edge length
    column4 = OneHot(is_edge_target)
    column5 = OneHot(is_edge_gasstation)

edgeStart and edgeEnd are the nodes located on the start and end of the edge, respectively.

In [493]:
Enviroment(poly.G, objects).norm_flagged_edges

array([[  0.   ,  11.   ,  52.473,   0.   ,   0.   ],
       [  0.   ,  10.   ,  72.482,   0.   ,   0.   ],
       [  0.   ,   1.   , 102.603,   0.   ,   0.   ],
       [  1.   ,   2.   , 110.083,   0.   ,   0.   ],
       [  1.   ,   0.   , 102.603,   0.   ,   0.   ],
       [  2.   ,  19.   ,  62.704,   0.   ,   1.   ],
       [  2.   ,   1.   , 110.083,   0.   ,   0.   ],
       [  3.   ,  15.   ,  27.169,   0.   ,   0.   ],
       [  4.   ,   0.   ,  67.433,   0.   ,   0.   ],
       [  4.   ,   5.   ,  27.143,   0.   ,   0.   ],
       [  5.   ,   4.   ,  27.143,   0.   ,   0.   ],
       [  6.   ,  18.   ,  20.45 ,   1.   ,   0.   ],
       [  7.   ,  16.   ,  55.913,   0.   ,   0.   ],
       [  8.   ,  16.   ,  46.158,   0.   ,   0.   ],
       [  8.   ,   1.   ,  76.243,   0.   ,   0.   ],
       [  9.   ,  10.   ,  18.803,   0.   ,   0.   ],
       [ 10.   ,   7.   ,   4.771,   0.   ,   0.   ],
       [ 10.   ,   9.   ,  18.803,   0.   ,   1.   ],
       [ 11.   ,   0.   ,  5

### Alternative Build

In [484]:
import networkx as nx

class Enviroment2():
    
    def __init__(self, G, objects):
        '''
        This alternative build of Enviroment leverages different data structures that are compatible with networkx.
        Utilizing these data structures, it is more easy to work with new GNN technologies, such as PyTorch Geometric
        which contains a from_networkx(G) function in its utils module. This has not yet been made compatible with 
        class State, primarily due to States dependancy on Enviroments norm_flagged_edges attribute. I am still working
        out a solution for this that best matches the goal of this class.
        
        params: 
                G:              MultiDiGraph object generated via polypocket.StreetMap.
                objects:        Dict object containing indexs of osmids that represent a house or gasstation
                
        attributes: 
                index:
                connections:    Returns array of shape(n, 2); [edgeStart, edgeEdge]
                nodes:          Returns dict(node_id= dict(house=bool, gasstation=bool))
                edges:          Returns list(tuple(edgeStart, edgeEnd, dict(attributes=values)))
                houses:         Returns array of shape(n, ); [house_node_id]
                gasstations:    Returns array of shape(n, ); [gasstation_node_id]
                G:              Returns cleaned MultiDiGraph
        
        methods: 
                get_neighbors(self, node_id): Returns array of shape (n, ); [neighbor_node_id]
        '''
        # Creates index dict for normalizing IDs
        def create_index(nodes):
            nodes = nodes.keys()
            nodes_index = list(range(len(nodes)))
            return dict(zip(nodes, nodes_index))   
    
        # Cleans nodes attributes
        def clean_nodes(nodes, objects):
            cleaned_nodes = dict()
            for node_id, _ in nodes.items():

                # Adds attributes to indicate house, gasstation
                house = 1 if node_id in objects.get('houses') else 0
                gasstation = 1 if node_id in objects.get('gasstations') else 0

                cleaned_nodes.update({
                    node_id : dict(house=house, 
                                   gasstation=gasstation)
                })

            return cleaned_nodes   
        
        # Standardizes with index
        def standardize_nodes(nodes, index):
            if index.keys() == nodes.keys():
                return dict(zip(index.values(), nodes.values()))
            else:
                raise AttributeError('Keys in nodes and index do not match.')
        
        # Standardizes with index
        def standardize_edges(edges, index):
            # Must be numeric/bool values to work
            keeps = [ 
                'oneway', 
                'length', 
                'speed_kph'
            ]
            
            # Reconstructs edges standardized and cleaned
            standard_edges = list()
            for node, connection, attributes in edges:
                keeps_values = list(map(attributes.get, keeps))
                standard_edges.append(
                    (index.get(node), index.get(connection), dict(zip(keeps, keeps_values)))
                )

            return standard_edges
        
        def standardize_connections(connections, index):
            stnd_connections = np.empty(shape=(connections.shape), dtype='int64')
            starts = np.array(list(map(index.get, connections[:,0]))).reshape(-1,1)
            ends   = np.array(list(map(index.get, connections[:,-1]))).reshape(-1,1)

            return np.append(starts, ends, axis=1)
        
        nodes = dict(G.nodes(data=True))
        edges =  G.edges(data=True)
        connections = np.array(list(edges))[:,:-1]
        
        # Enviroment nodes, edges
        self.index = create_index(nodes)
        self.connections = standardize_connections(connections, index)
        self.nodes = clean_nodes(nodes, objects)
        self.nodes = standardize_nodes(self.nodes, self.index)
        self.edges = standardize_edges(edges, index)  
        
        # Enviroment houses and Gasstations
        self.houses      = np.array(list(map(index.get, objects.get('houses'))))
        self.gasstations = np.array(list(map(index.get, objects.get('gasstations'))))
        
        # Creates networkx graph from standardized data
        G = nx.MultiDiGraph()
        G.add_nodes_from(self.nodes)
        G.add_edges_from(self.edges)
        self.G = G
        
    # Returns available edges from a node
    def get_neighbors(self, node_id):   
        return self.connections[self.connections[:,0] == node_id][:,-1]